In [1]:
import os

In [2]:
%pwd

'/home/lucky/Documents/data_science_interview/Udemy/My_Projects/chicken_binary/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/lucky/Documents/data_science_interview/Udemy/My_Projects/chicken_binary'

In [5]:
ls

artifacts/  dvc.yaml  params.yaml       research/  template.py
config/     LICENSE   README.md         setup.py   templates/
data.zip    logs/     requirements.txt  src/       venv/


# Entity

In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    data_dir: Path
    csv_dir: Path

# Config Manager

In [20]:
from cnnClassifier.utils.common import create_directories, read_yaml
from cnnClassifier.constants import *

In [21]:
class ConfigurationManager:
    """
        responsible for creating the Directories needed for data
        and return entity -> DataIngestionConfig
    """

    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir,
            data_dir = config.data_dir,
            csv_dir = config.csv_dir
        )

        return data_ingestion_config

# Data Ingestion Class

In [22]:
import ssl
import urllib.request as request
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
import zipfile
import subprocess

In [23]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        # Create an unverified SSL context
        ssl._create_default_https_context = ssl._create_unverified_context
        
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [24]:

config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)
data_ingestion.download_file()
data_ingestion.extract_zip_file()

[2023-12-09 16:20:22,298: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-09 16:20:22,299: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-09 16:20:22,300: INFO: common: created directory at: artifacts]
[2023-12-09 16:20:22,301: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-09 16:22:02,866: INFO: 1943290467: artifacts/data_ingestion/data.zip download! with following info: 
Date: Sat, 09 Dec 2023 10:50:23 GMT
Content-Type: application/zip
Transfer-Encoding: chunked
Connection: close
Access-Control-Allow-Origin: *
Access-Control-Allow-Headers: Cache-Control,Authorization,Content-Type,X-reqed-With,x-requested-with
Access-Control-Allow-Methods: GET,HEAD,POST,PUT,PATCH,DELETE
Access-Control-Expose-Headers: X-RateLimit-Limit,X-RateLimit-Remaining,X-RateLimit-Reset
Retry-After: 1
X-RateLimit-Limit: 1
X-RateLimit-Remaining: 7
X-RateLimit-Reset: 2023-12-09T10:50:23.968Z
Content-disposition: attachment;filename=Chicken_fecal_d